# Summarize QA/QC Results 

Create summary tables from QAQC-PFRA notebook scraps in one or more folders on Amazon Web Services' S3.

### _Development Notebook_

### Import libraries

In [1]:
import sys;sys.path.append('../')
import gdal
from hecrasio.core import *
from hecrasio.qaqc import *
from hecrasio.s3tools import *
import scrapbook as sb
import pandas as pd
import shutil

### Set pandas display options

In [2]:
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 1000)

### Identify notebooks on S3

In [3]:
# Either of these will work

books = pull_scraps(single_folder = "s3://azavea/jwx_test/data/pfra_test_nbs")

# books = pull_scraps(multi_folder = 'yes', bucket = 'azavea', prefix = 'jwx_test/nbs/nb0', name_selector = 'DC')

In [59]:
scraps

Scraps([('{"Results":{"Base Output Interval":"15MIN","Computation Time Courant Method":"Representative Length\\/Velocity","Computation Time Step Base":"15SEC","Computation Time Step Count To Double":2,"Computation Time Step Max Courant":2.0,"Computation Time Step Max Doubling":2,"Computation Time Step Max Halving":1,"Computation Time Step Min Courant":0.5,"Flow Filename":"DC_F02_NBR.u01","Flow Title":"E0097","Geometry Filename":"DC_F02_NBR.g01","Geometry Title":"D01","Plan Filename":"DC_F02_NBR.p01","Plan Name":"Extreme_event","Plan ShortID":"Extreme_event","Plan Title":"Extreme_event","Simulation End Time":"01May2000 16:00:0","Simulation Start Time":"01May2000 10:00:0","Time Window":"01May2000 10:00:00 to 01May2000 16:00:0"}}',
         Scrap(name='{"Results":{"Base Output Interval":"15MIN","Computation Time Courant Method":"Representative Length\\/Velocity","Computation Time Step Base":"15SEC","Computation Time Step Count To Double":2,"Computation Time Step Max Courant":2.0,"Computat

In [57]:
if list(scraps)[-1] == 'Global Errors':
    print("The following Global Errors occured: {}".format(scraps.data_scraps['Global Errors'].data))

The following Global Errors occured: ['Failed to Show Results', 'Failed to glue Results Data', 'Failed to glue Plan Summary']


In [69]:
books[7][1].scraps

Scraps([('{"Results":{"Base Output Interval":"15MIN","Computation Time Courant Method":"Representative Length\\/Velocity","Computation Time Step Base":"15SEC","Computation Time Step Count To Double":2,"Computation Time Step Max Courant":2.0,"Computation Time Step Max Doubling":2,"Computation Time Step Max Halving":1,"Computation Time Step Min Courant":0.5,"Flow Filename":"DC_F02_NBR.u01","Flow Title":"E0097","Geometry Filename":"DC_F02_NBR.g01","Geometry Title":"D01","Plan Filename":"DC_F02_NBR.p01","Plan Name":"Extreme_event","Plan ShortID":"Extreme_event","Plan Title":"Extreme_event","Simulation End Time":"01May2000 16:00:0","Simulation Start Time":"01May2000 10:00:0","Time Window":"01May2000 10:00:00 to 01May2000 16:00:0"}}',
         Scrap(name='{"Results":{"Base Output Interval":"15MIN","Computation Time Courant Method":"Representative Length\\/Velocity","Computation Time Step Base":"15SEC","Computation Time Step Count To Double":2,"Computation Time Step Max Courant":2.0,"Computat

In [137]:
result_dict = {}
for tup in books:
    nb, results = tup
    scrap_data = []
    for scrap in results.scraps:
        if scrap == 'Global Errors':
            errors = results.scraps['Global Errors'].data
            print("WARNING! {} had the following global errors: {}".format(nb, errors))
        elif scrap != 'Global Errors':
            logged_information = list(json.loads(scrap).values())[0]
            scrap_data.append(logged_information)
        else:
            print('Unexpected scrap name identified, please troubleshoot!')
    result_dict[nb] = dict(ChainMap(*scrap_data))

WARNING! DC_F02_NBR_E0029_out had the following global errors: ["Check files...too many hdf's found", 'Failed to read RAS plan: rasPlan error', 'Failed to glue Plan Information', 'Failed to glue Plan Parameters', 'Failed to Show Results', 'Failed to glue Results Data', 'Failed to glue Plan Summary']
WARNING! DC_F02_NBR_E0097_out had the following global errors: ['Failed to Show Results', 'Failed to glue Results Data', 'Failed to glue Plan Summary']


In [138]:
pd.DataFrame.from_dict(result_dict).T

,1D Cores,1D Methodology,1D2D Flow Tolerance,1D2D MaxIter,1D2D Minimum Flow Tolerance,1D2D WS Tolerance,2D Boundary Condition Ramp Up Fraction,2D Boundary Condition Volume Check,2D Cores,2D Coriolis,2D Eddy Viscosity Transverse Mixing Coefficient,2D Equation Set,2D Initial Conditions Ramp Up Time (hrs),2D Latitude for Coriolis,2D Maximum Iterations,2D Names,2D Number of Time Slices,2D Only,2D Theta,2D Theta Warmup,2D Volume Tolerance,2D Water Surface Tolerance,Base Output Interval,Computation Time Courant Method,Computation Time DSS,Computation Time Step Base,Computation Time Step Count To Double,Computation Time Step Max Courant,Computation Time Step Max Doubling,Computation Time Step Max Halving,Computation Time Step Min Courant,Computation Time Total,Flow Filename,Flow Title,Geometry Filename,Geometry Title,HDF Chunk Size,HDF Compression,HDF Fixed Rows,HDF Flush Buffer,HDF Spatial Parts,HDF Use Max Rows,HDF Write Face Node Velocities,HDF Write Time Slices,HDF Write Warmup,Instability Count,Max Velocity,Maximum WSEL Error,Pardiso Solver,Plan Filename,Plan Name,Plan ShortID,Plan Title,Run Time Window,Simulation End Time,Simulation Start Time,Solution,Time Solution Went Unstable,Time Stamp Solution Went Unstable,Time Window,Vol Accounting,Vol Accounting 2D Bdry,Vol Accounting Cell End,Vol Accounting Cell Start,Vol Accounting DS Outflow,Vol Accounting Error,Vol Accounting Error Percentage,Vol Accounting Groundwater,"Vol Accounting LS, Pump Div",Vol Accounting Lat Hydro,Vol Accounting US Inflow
ARLAMS,0,Finite Difference,0.1,0,1,0.01,"[0.1000000015, 0.1000000015, 0.1000000015, 0.1...","[False, False, False, False, False]",0,False,"[3.402823466e+38, 3.402823466e+38, 3.402823466...","[Dynamic Wave, Dynamic Wave, Dynamic Wave, Dyn...","[3.402823466e+38, 3.402823466e+38, 3.402823466...","[3.402823466e+38, 3.402823466e+38, 3.402823466...","[20, 20, 20, 20, 20]","[Area1, Area2, Area3, Area4, Area5]","[1, 1, 1, 1, 1]",True,"[1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0]","[0.0099999998, 0.0099999998, 0.0099999998, 0.0...","[0.0099999998, 0.0099999998, 0.0099999998, 0.0...",1HOUR,NaN,00:00:00,30SEC,NaN,NaN,NaN,NaN,NaN,02:36:45,ARLAMS.u01,Precip_Excess,ARLAMS.g01,ARLAMS_Pluvial,1,1,1,False,1,0,False,False,False,0,13.1837,0,False,ARLAMS.p01,reducedPluvial_E005,reducedPluvial_E005,reducedPluvial_E005,04JAN2019 20:04:14 to 04JAN2019 22:40:54,05May2018 00:00:00,01May2018 00:00:00,Unsteady Finished Successfully,00:00:00,Not Applicable,01May2018 00:00:00 to 05May2018 00:00:00,Volume Accounting in Acre Feet,0,662727,0,0,854.456,0.119748,0,0,0,0
DC_F01_B01,0,Finite Difference,0.1,0,1,0.01,[0.1000000015],[False],0,False,[3.402823466e+38],[Dynamic Wave],[3.402823466e+38],[3.402823466e+38],[20],[D01],[1],True,[1.0],[1.0],[0.0099999998],[0.0099999998],15MIN,Representative Length/Velocity,00:00:00,15SEC,2,2,2,1,0.5,03:41:13,DC_F01_B01.u01,Extreme_event,DC_F01_B01.g01,D01,1,1,1,False,1,0,False,False,False,3477,96671.7,0,False,DC_F01_B01.p01,Extreme_event,Extreme_event,Extreme_event,21AUG2019 17:24:25 to 21AUG2019 21:05:37,05May1996 00:00:00,01May1996 00:00:00,Unsteady Finished Successfully,00:00:00,Not Applicable,01May1996 00:00:00 to 05May1996 00:00:00,Volume Accounting in Acre Feet,0,106923,87092.2,0,-95.1274,0.00205718,0,0,0,0
DC_F01_NBR_E0001_out,0,Finite Difference,0.1,0,1,0.01,[0.1000000015],[False],0,False,[3.402823466e+38],[Dynamic Wave],[3.402823466e+38],[3.402823466e+38],[20],[D01],[1],True,[1.0],[1.0],[0.0099999998],[0.0099999998],15MIN,Representative Length/Velocity,00:00:00,30SEC,2,2,2,1,0.5,03:55:48,DC_F01_NBR.u01,E0001,DC_F01_NBR.g01,D01,1,1,1,False,1,0,False,False,False,6824,37015.7,0,False,DC_F01_NBR.p01,test_scaling,test_scaling,test_scaling,15JUL2019 17:03:30 to 15JUL2019 20:59:17,05May2000 10:00:0,01May2000 10:00:0,Unsteady Finished Successfully,00:00:00,Not Applicable,01May2000 10:00:00 to 05May2000 10:00:0,Volume Accounting in Acre Feet,0,107467,84515.9,0,-1.9559,4.23324e-05,0,0,0,0
DC_F01_NBR_E0001_p01,0,Finite Difference

### Create QA/QC results table

In [4]:
results = make_qaqc_table(books)
results.head(5)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Create unique values summary table
**Not implimented:** If length of unque value list is equal to a single value, ignore attribute and return that all values are the same.

In [ ]:
relevant_columns = ['Base Output Interval',
                    '2D Equation Set',
                    '2D Volume Tolerance',
                    '2D Latitude for Coriolis',
                    '2D Water Surface Tolerance',
                    'Solution',
                    'Vol Accounting Error',
                    'Max Velocity',
                    'Instability Count']

unique_df = identify_unique_values(results, relevant_columns)

unique_df.head(10)

### Create threshold summary table

In [ ]:
create_summary_table(unique_df, results)

# END